In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [40]:
Historical_DOU_NCH = pd.read_csv("2020 DOU Ambulatory NCH.csv")
Q1 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([1,2,3])]
Q2 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([4,5,6])]
Q3 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([7,8,9])]
Q4 = Historical_DOU_NCH[Historical_DOU_NCH.Month.isin([10,11,12])]

In [3]:
Amb_NCH_Data = pd.read_csv("2021_Q2 Ambulatory NCH.csv")

In [4]:
set(Amb_NCH_Data.Hospital)

{'EDH', 'EHH', 'EJCH', 'ESJH', 'EUH', 'EUHM'}

In [5]:
Amb_NCH_Data.head()

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
0,EJCH,JC CATH,5,1128.61,0.00,0.00,0,0.00,0.0
1,EJCH,JC CATH,6,918.82,0.00,0.00,0,0.00,0.0
2,EJCH,JC CATH,4,1049.28,0.00,0.00,0,0.00,0.0
3,EJCH,JC CDU,6,1211.66,208.68,0.00,0,700.43,0.0
4,EJCH,JC CDU,4,1164.21,174.68,94.46,0,471.14,0.0


In [6]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_Ambulatory_NCH_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to update the Ambulatory Nursing Care Hours

    # The first step takes only the data for this selected hospital
    Hosp_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct NDNQI code, year, and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # Take the xml UnitName and replace it with the correct EJCH UnitName from the EJCH_Unit list. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]

        # Now grab each of the 3 months for that unit from the Hospital data
        new_data = Hosp_data[Hosp_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        new_data = new_data.sort_values('Month')

        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = new_data.Month.iloc[idx2]

            Amb_NCH_nodes = month.getElementsByTagName("NursingCareHoursAmbulatory")
            for idx3, Amb_NCH_node in enumerate(Amb_NCH_nodes):

                for column in new_data.columns[3:]:
                    new_value = new_data[column].iloc[idx2]
                    Amb_NCH_node.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value
                    

    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} Ambulatory NCH.xml'.format(Calendar_Quarter,Calendar_Year,Selected_Hosp), 'w'))

## EJCH

In [7]:
NDNQI_Ambulatory_NCH_XML('EJCH','2020 EJCH Amb_NCH Template.xml','YTD069',2021,2)

In [8]:
EJCH_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == 'EJCH']
# A List of this Hospital's Unit Names
EJCH_Units = list(set(EJCH_data.UnitName))

In [9]:
EJCH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
0,EJCH,JC CATH,5,1128.61,0.00,0.00,0,0.00,0.0
1,EJCH,JC CATH,6,918.82,0.00,0.00,0,0.00,0.0
2,EJCH,JC CATH,4,1049.28,0.00,0.00,0,0.00,0.0
3,EJCH,JC CDU,6,1211.66,208.68,0.00,0,700.43,0.0
4,EJCH,JC CDU,4,1164.21,174.68,94.46,0,471.14,0.0
5,EJCH,JC CDU,5,1372.55,165.18,50.68,0,759.80,0.0
6,EJCH,JC ROCU,5,1560.84,0.27,0.00,0,0.00,0.0
7,EJCH,JC ROCU,6,1407.07,0.00,0.00,0,0.00,0.0
8,EJCH,JC ROCU,4,1452.73,14.64,0.00,0,0.00,0.0
9,EJCH,JCEndo,4,987.66,0.00,0.00,0,0.00,0.0


## ESJH

In [10]:
NDNQI_Ambulatory_NCH_XML('ESJH','2020 ESJH Amb_NCH Template.xml','XAR317',2021,2)

In [11]:
ESJH_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == 'ESJH']
# A List of this Hospital's Unit Names
ESJH_Units = list(set(ESJH_data.UnitName))

In [12]:
ESJH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
15,ESJH,1 West (Extended observation unit),6,1416.89,166.07,0.0,0,525.07,0.00
16,ESJH,1 West (Extended observation unit),4,1391.29,135.12,0.0,0,471.74,5.09
17,ESJH,1 West (Extended observation unit),5,1382.06,110.79,0.0,0,465.14,10.75
18,ESJH,ARU/DRU Procedure Prep/Recovery,4,2380.35,0.00,0.0,0,930.81,0.00
19,ESJH,ARU/DRU Procedure Prep/Recovery,6,2275.93,0.00,0.0,0,787.72,0.00
20,ESJH,ARU/DRU Procedure Prep/Recovery,5,2556.00,0.54,0.0,0,912.28,0.00
21,ESJH,ESJH Cardiac Cath Lab,4,812.47,874.59,0.0,0,176.54,0.00
22,ESJH,ESJH Cardiac Cath Lab,5,902.14,675.81,0.0,0,179.05,0.00
23,ESJH,ESJH Cardiac Cath Lab,6,843.79,882.93,0.0,0,105.54,0.00
24,ESJH,ESJH Cardiac Rehab/Screening,4,118.37,0.00,0.0,0,0.00,0.00


## EUH

In [13]:
NDNQI_Ambulatory_NCH_XML('EUH','2020 EUH Amb_NCH Template.xml','JXE432',2021,2)

In [14]:
EUH_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == 'EUH']
# A List of this Hospital's Unit Names
EUH_Units = list(set(EUH_data.UnitName))

In [15]:
EUH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
72,EUH,EUH Cardiac Cath Lab,4,1706.11,2.04,0.0,0,0.00,0.00
73,EUH,EUH Cardiac Cath Lab,5,1706.32,13.61,0.0,0,0.00,0.00
74,EUH,EUH Cardiac Cath Lab,6,1590.10,39.53,0.0,0,0.00,0.00
75,EUH,EUH Cardiac Echo,5,699.23,105.22,0.0,0,0.00,0.00
76,EUH,EUH Cardiac Echo,4,655.35,82.64,0.0,0,0.00,0.00
77,EUH,EUH Cardiac Echo,6,655.20,334.11,0.0,0,0.00,0.00
78,EUH,EUH Dialysis,6,2221.57,359.39,0.0,0,0.00,0.00
79,EUH,EUH Dialysis,5,2336.89,359.36,0.0,0,0.00,0.00
80,EUH,EUH Dialysis,4,2205.57,442.52,0.0,0,0.00,0.00
81,EUH,EUH Endoscopy,6,931.50,366.32,0.0,0,332.11,0.00


## EUHM

In [16]:
NDNQI_Ambulatory_NCH_XML('EUHM','2020 EUHM Amb_NCH Template.xml','PVF155',2021,2)

In [17]:
EUHM_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == 'EUHM']
# A List of this Hospital's Unit Names
EUHM_Units = list(set(EUHM_data.UnitName))

In [18]:
EUHM_data.head()

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
45,EUHM,EUHM Arrythmia Center,4,459.98,61.75,0.0,0,0.0,0.0
46,EUHM,EUHM Arrythmia Center,5,584.11,58.92,0.0,0,0.0,0.0
47,EUHM,EUHM Arrythmia Center,6,501.70,33.15,0.0,0,0.0,0.0
48,EUHM,EUHM Cardiac Echo,5,466.05,24.57,0.0,0,0.0,0.0
49,EUHM,EUHM Cardiac Echo,4,451.17,0.00,0.0,0,0.0,0.0


## EDH

In [19]:
NDNQI_Ambulatory_NCH_XML('EDH','2021 EDH Amb_NCH Template.xml','NMW346',2021,2)

In [20]:
EDH_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == 'EDH']
# A List of this Hospital's Unit Names
EDH_Units = list(set(EDH_data.UnitName))

In [21]:
EDH_data.head()

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
93,EDH,EDH 12,6,980.72,397.46,0.0,0,212.25,0.0
94,EDH,EDH 12,5,805.59,412.86,0.0,0,248.62,0.0
95,EDH,EDH 12,4,769.76,513.47,0.0,0,208.86,0.0
96,EDH,EDH Cath Lab,5,332.86,866.80,0.0,0,0.00,0.0
97,EDH,EDH Cath Lab,4,300.69,579.16,0.0,0,0.00,0.0


## EHH

In [22]:
NDNQI_Ambulatory_NCH_XML('EHH','2021 EHH Amb_NCH Template.xml','WXX116',2021,2)

In [23]:
EHH_data = Amb_NCH_Data[Amb_NCH_Data.Hospital == 'EHH']
# A List of this Hospital's Unit Names
EHH_Units = list(set(EHH_data.UnitName))

In [24]:
EHH_data

,Hospital,UnitName,Month,RNHospEmplHours,RNContractEmplHours,LPNHospEmplHours,LPNContractEmplHours,UAPHospEmplHours,UAPContractEmplHours
114,EHH,EHH Interventional Procedures,5,485.55,53.04,0.0,0,0.00,0.0
115,EHH,EHH Interventional Procedures,4,485.41,0.00,0.0,0,10.25,0.0
116,EHH,EHH Interventional Procedures,6,498.68,42.17,0.0,0,0.00,0.0
